In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import glob
import os
import numpy as np

In [ ]:
region_gdf = gpd.read_file('../clip/brazil_munis_interviews_aea.shp')

In [ ]:
all_csvs = glob.glob('./out/*.csv')
all_csvs.sort()

In [ ]:
def read_process_csv(csv):
    temp_df = pd.read_csv(csv)
    temp_df['satellite'] = os.path.basename(csv)[:8]
    temp_df['year'] = int(os.path.basename(csv)[9:13])
    return temp_df

In [ ]:
full_df = pd.concat([
    read_process_csv(csv) for csv in all_csvs
])

In [ ]:
full_df['geometry'] = gpd.points_from_xy(full_df['center_x'], full_df['center_y'], crs='ESRI:102033')

In [ ]:
    full_gdf = gpd.GeoDataFrame(full_df, geometry = full_df['geometry'])

In [ ]:
# full_gdf = full_gdf.loc[~((full_gdf['satellite'] != 'landsat8') & (full_gdf['year']>=2014))]
# full_gdf = full_gdf.loc[~((full_gdf['satellite']=='landsat5') & (full_gdf['year']>=2000))]

In [ ]:
full_gdf = full_gdf.loc[(full_gdf['satellite'] != 'landsat8')]
full_gdf = full_gdf.loc[~((full_gdf['satellite']=='landsat5') & (full_gdf['year']>=2000))]

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(15,15))
region_gdf.boundary.plot(ax=ax)
xlims = ax.get_xlim()
ylims = ax.get_ylim()
ylims_range = ylims[1] - ylims[0]
xlims_range = xlims[1] - xlims[0]
full_gdf.loc[full_gdf['year']==2011].plot(kind='hexbin',
                                          x='center_x', 
                                          y='center_y', 
                                          gridsize=(10, int(10*ylims_range/xlims_range)),  
                                          cmap='viridis', ax=ax)


In [ ]:
xlims = region_gdf.bounds.min()['minx'], region_gdf.bounds.max()['maxx']
ylims = region_gdf.bounds.min()['miny'], region_gdf.bounds.max()['maxy']
ylims_range = ylims[1] - ylims[0]
xlims_range = xlims[1] - xlims[0]
grid_size = (12, int(12*ylims_range/xlims_range))

In [ ]:
# xlims = np.min(full_df['center_x']), np.max(full_df['center_x'])
# ylims =  np.min(full_df['center_y']), np.max(full_df['center_y'])
# ylims_range = ylims[1] - ylims[0]
# xlims_range = xlims[1] - xlims[0]
# grid_size = (10, int(10*ylims_range/xlims_range))

In [ ]:
%matplotlib notebook

fig, ax = plt.subplots(figsize=(8,16))
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are offregion_gdf.boundary.plot(ax=ax, color='red')

ax.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    right=False,      # ticks along the bottom edge are off
    left=False,         # ticks along the top edge are off
    labelleft=False) # labels along the bottom edge are offregion_gdf.boundary.plot(ax=ax, color='red')

region_gdf.boundary.plot(ax=ax, color='red')

year_start = 1984
year_end = 2020

def animate(i):
    df = full_gdf.loc[full_gdf['year']==i]
    x = df['center_x'].values
    y = df['center_y'].values
    if i != year_start:
        ax.collections[-1].remove()
    hexlayer = ax.hexbin(x,y, gridsize=grid_size,
                         alpha=1.0,
                        vmin=0, vmax=50,
                        extent=(xlims[0], xlims[1], ylims[0], ylims[1]))
    ax.set_title(i)
    return hexlayer,

ani = FuncAnimation(fig, animate, frames=np.arange(year_start, year_end), interval=500)
ani.save('test_animation_count.gif')

In [ ]:
%matplotlib notebook

fig, ax = plt.subplots(figsize=(8,16))
ax.set_xlim(xlims)
ax.set_ylim(ylims)
region_gdf.boundary.plot(ax=ax, color='red')

year_start = 1984
year_end = 2023

def animate(i):
    df = full_gdf.loc[full_gdf['year']==i]
    x = df['center_x'].values
    y = df['center_y'].values
    c = df['area'].values
    if i != year_start:
        ax.collections[-1].remove()
    hexlayer = ax.hexbin(x,y, c,reduce_C_function=np.sum, gridsize=grid_size,
                         alpha=1.0,
                        vmin=0, vmax=10000,
                        extent=(xlims[0], xlims[1], ylims[0], ylims[1]))
    ax.set_title(i)
    return hexlayer,

ani = FuncAnimation(fig, animate, frames=np.arange(year_start, year_end), interval=500)
ani.save('test_animation_area.gif')

In [ ]:
# %matplotlib notebook
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.animation import FuncAnimation

# fig, ax = plt.subplots()
# xdata, ydata = [], []
# ln, = ax.plot([], [], 'ro')
# time_text = ax.text(1, 1, '', fontsize=15)

# def init():
#     ax.set_xlim(full_gdf['center_x'].min(), full_gdf['center_x'].max())
#     ax.set_ylim(full_gdf['center_y'].min(), full_gdf['center_y'].max())
#     return ln,

# def update(frame):
#     df = full_gdf.loc[full_gdf['year']==frame]
#     xdata = df['center_x'].values
#     ydata = df['center_y'].values
#     zdata = df['area'].values
#     ln.set_data(xdata, ydata)
#     time_text.set_text(frame)
#     print(frame)
#     ln.set_ms(5)
#     return ln, time_text

# ani = FuncAnimation(fig, update, frames=full_gdf['year'].unique(),
#                     init_func=init, blit=False)
# plt.show()


# Try making gif of landcover

In [ ]:
all_lc_tifs = glob.glob('../clip/out/')